In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import sys
from MetodosNumericos import MetodosNumericos

from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import statsmodels.formula.api as sm

# Practica 

In [2]:
data = pd.read_csv('../data/punto_4/base_p4.csv', index_col=0)
data.head()

,faminc,cigtax,cigprice,bwght,fatheduc,motheduc,parity,male,white,cigs,lbwght,bwghtlbs,packs,lfaminc
1,13.5,16.5,122.300003,109,12.0,12.0,1,1,1,0,4.691348,6.8125,0.0,2.602690
2,7.5,16.5,122.300003,133,6.0,12.0,2,1,0,0,4.890349,8.3125,0.0,2.014903
3,0.5,16.5,122.300003,129,NaN,12.0,2,0,0,0,4.859812,8.0625,0.0,-0.693147
4,15.5,16.5,122.300003,126,12.0,12.0,2,1,0,0,4.836282,7.8750,0.0,2.740840
5,27.5,16.5,122.300003,134,14.0,12.0,2,1,1,0,4.897840,8.3750,0.0,3.314186


In [6]:
data.shape

(1387, 6)

In [3]:
# Quitar nulos
data = data.drop('fatheduc', axis=1)
data = data.dropna(subset=['motheduc'])

In [4]:
# Filtrar las columnas mas significativas según el EDA
data = data[['faminc', 'cigprice', 'bwght', 'motheduc', 'male', 'cigs']]

# Método Gauss

In [5]:
data1 = data.copy()

# Agregar una columna de unos para el término de intersección
interc = np.ones((len(data1), 1))
data1['intercepto'] = interc


# Separar las variables independientes (X) de la variable dependiente (y)
datos_prueba_X = data1.drop('bwght', axis=1)
A = datos_prueba_X.T.dot(datos_prueba_X)
b = datos_prueba_X.T.dot(data1['bwght'])

# Crear una instancia de la clase MetodosNumericos
mn = MetodosNumericos()

#Aplicar método de Gauss
coefs = mn.backward_substi(mn.Gauss(A.to_numpy(),b.to_numpy())[0], mn.Gauss(A.to_numpy(),b.to_numpy())[1])

# Obtener los nombres de las variables
nombres_variables = datos_prueba_X.columns.tolist()

# Imprimir los resultados
print("=" * 23) 
print("{:<12} {:>10} ".format("Variable", "Coef"))
print("-" * 23) 

for nombre, coeficiente in zip(nombres_variables, coefs):
    print("{:<12} {:>10.15f} ".format(nombre, coeficiente))


Variable           Coef 
-----------------------
faminc       0.093027428362224 
cigprice     0.078669291307812 
motheduc     -0.017912397883051 
male         3.034044821097501 
cigs         -0.466343700753196 
intercepto   105.359264957150586 


In [17]:
data1 = data.copy()

# Agregar una columna de unos para el término de intersección
interc = np.ones((len(data1), 1))
data1['intercepto'] = interc


# Separar las variables independientes (X) de la variable dependiente (y)
datos_prueba_X = data1.drop('bwght', axis=1)
A = datos_prueba_X.T.dot(datos_prueba_X)
b = datos_prueba_X.T.dot(data1['bwght'])

# Crear una instancia de la clase MetodosNumericos
mn = MetodosNumericos()

#Aplicar método de Gauss
coefs = mn.backward_substi(mn.Gauss(A.to_numpy(),b.to_numpy())[0], mn.Gauss(A.to_numpy(),b.to_numpy())[1])

# Obtener los nombres de las variables
nombres_variables = datos_prueba_X.columns.tolist()

# Step 2: Calculate Residuals
predicted_y = datos_prueba_X.dot(coefs)
residuals = data1['bwght'] - predicted_y

# Step 3: Calculate Total Sum of Squares (TSS)
TSS = np.sum((data1['bwght'] - np.mean(data1['bwght'])) ** 2)

# Step 4: Calculate Residual Sum of Squares (RSS)
RSS = np.sum(residuals ** 2)

# Step 5: Calculate R^2
R_squared = 1 - (RSS / TSS)

# Step 6: Calculate Standard Errors
# Compute variance-covariance matrix of coefficient estimates
n = len(data1)
p = len(coefs)
X = datos_prueba_X.to_numpy()
sigma_squared = RSS / (n - p)
var_cov_matrix = np.linalg.inv(A) * sigma_squared

# Standard errors are square roots of the diagonal elements of the variance-covariance matrix
standard_errors = np.sqrt(np.diag(var_cov_matrix))

# Imprimir los resultados
print("=" * 54) 
print("{:<12} {:>20} {:>20}".format("Variable", "Coef", "Standard Error"))
print("-" * 54) 

for nombre, coeficiente, error in zip(nombres_variables, coefs, standard_errors):
    print("{:<12} {:>20.15f} {:>20.15f}".format(nombre, coeficiente, error))

print("=" * 54)
print("R-squared:", R_squared)


Variable                     Coef       Standard Error
------------------------------------------------------
faminc          0.093027428362224    0.015550346981774
cigprice        0.078669291307812    0.007602766599825
motheduc       -0.017912397883051    0.237225027132588
male            3.034044821097501    1.077710632851107
cigs           -0.466343700753196    0.092231646784815
intercepto    105.359264957150586    7.393011191506387
R-squared: 0.037094795842924966


In [18]:
import numpy as np

class RegressionMetrics:
    def __init__(self, data, var_objetivo):
        self.data = data.copy()
        self.var_objetivo = var_objetivo
        
    def fit(self):
        # intersección
        intercept = np.ones((len(self.data), 1))
        self.data['intercept'] = intercept

        # Separar las variables independientes (X) de la variable dependiente (y)
        X = self.data.drop(self.var_objetivo, axis=1)
        y = self.data[self.var_objetivo]

        # Calcular la matriz de diseño
        A = X.T.dot(X)
        b = X.T.dot(y)

        # Aplicar el método de Gauss
        mn = MetodosNumericos()
        coefs = mn.backward_substi(mn.Gauss(A.to_numpy(), b.to_numpy())[0], mn.Gauss(A.to_numpy(), b.to_numpy())[1])

        # Calcular los nombres de las variables
        self.variable_names = X.columns.tolist()

        # Calcular las predicciones
        self.predicted_y = X.dot(coefs)

        # Calcular los residuos
        self.residuals = y - self.predicted_y

        # Calcular el Total Sum of Squares (TSS)
        self.TSS = np.sum((y - np.mean(y)) ** 2)

        # Calcular el Residual Sum of Squares (RSS)
        self.RSS = np.sum(self.residuals ** 2)

        # Calcular R-squared
        self.R_squared = 1 - (self.RSS / self.TSS)

        # Calcular los errores estándar
        n = len(self.data)
        p = len(coefs)
        X_matrix = X.to_numpy()
        sigma_squared = self.RSS / (n - p)
        var_cov_matrix = np.linalg.inv(A) * sigma_squared
        self.standard_errors = np.sqrt(np.diag(var_cov_matrix))

    def print_metrics(self):
        # Imprimir los resultados
        print("=" * 54) 
        print("{:<12} {:>20} {:>20}".format("Variable", "Coef", "Standard Error"))
        print("-" * 54) 

        for name, coef, error in zip(self.variable_names, coefs, self.standard_errors):
            print("{:<12} {:>20.15f} {:>20.15f}".format(name, coef, error))

        print("=" * 54)
        print("R-squared:", self.R_squared)

# Uso de la clase
if __name__ == "__main__":
    # Crear una instancia de RegressionMetrics
    regression = RegressionMetrics(data, 'bwght')
    regression.fit()
    regression.print_metrics()


Variable                     Coef       Standard Error
------------------------------------------------------
faminc          0.093027428362224    0.015550346981774
cigprice        0.078669291307812    0.007602766599825
motheduc       -0.017912397883051    0.237225027132588
male            3.034044821097501    1.077710632851107
cigs           -0.466343700753196    0.092231646784815
intercept     105.359264957150586    7.393011191506387
R-squared: 0.037094795842924966


In [11]:
import numpy as np

# Step 2: Calculate Residuals
predicted_y = datos_prueba_X.dot(coefs)
residuals = data1['bwght'] - predicted_y

# Step 3: Calculate Total Sum of Squares (TSS)
TSS = np.sum((data1['bwght'] - np.mean(data1['bwght'])) ** 2)

# Step 4: Calculate Residual Sum of Squares (RSS)
RSS = np.sum(residuals ** 2)

# Step 5: Calculate R^2
R_squared = 1 - (RSS / TSS)

# Step 6: Calculate Standard Errors
# Compute variance-covariance matrix of coefficient estimates
n = len(data1)
p = len(coefs)
X = datos_prueba_X.to_numpy()
sigma_squared = RSS / (n - p)
var_cov_matrix = np.linalg.inv(A) * sigma_squared

# Standard errors are square roots of the diagonal elements of the variance-covariance matrix
standard_errors = np.sqrt(np.diag(var_cov_matrix))


In [12]:
R_squared

0.037094795842924966

# Skelearn

In [19]:
data2 = data.copy()

# # Separar las variables independientes (X) de la variable dependiente (y)
x = data.drop('bwght', axis=1)
y = data[['bwght']]

# Crear una instancia de LinearRegression
modelo = LinearRegression()

# Ajustar el modelo a los datos
modelo.fit(x, y)

# Obtener los coeficientes y el término de intersección
coeficientes = modelo.coef_
intercepto = modelo.intercept_

# Obtener los nombres de las variables
nombres_variables = x.columns.tolist()

print("=" * 23)
print("Coeficientes de regresión lineal:")
print("-" * 23)
for nombre, coeficiente in zip(nombres_variables, coeficientes[0]):
    print("{:<12}: {:>10.15f}".format(nombre, coeficiente))
print("-" * 30)
print("intercepto:", intercepto)
print("=" * 30)

Coeficientes de regresión lineal:
-----------------------
faminc      : 0.093027428362222
cigprice    : 0.078669291307840
motheduc    : -0.017912397883037
male        : 3.034044821097491
cigs        : -0.466343700753196
------------------------------
intercepto: [105.35926496]


# Stastmodels

In [11]:
data3 = data.copy()
formula = 'bwght ~ faminc + cigprice + motheduc + male + cigs'
model = sm.ols(formula = formula, data = data3)
fitted = model.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     10.64
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           4.77e-10
Time:                        08:41:32   Log-Likelihood:                -6121.1
No. Observations:                1387   AIC:                         1.225e+04
Df Residuals:                    1381   BIC:                         1.229e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    105.3593      7.393     14.251      0.0